In [1]:
%load_ext autoreload
%autoreload 2

In [1]:
import os
import pandas as pd
import numpy as np
import logging
import joblib
import pickle
import lmdb
from Bio import PDB
from Bio.PDB import PDBExceptions
from torch.utils.data import Dataset
from tqdm.auto import tqdm

import torch
import matplotlib.pyplot as plt
import seaborn as sns

import random

from Bio.PDB import PDBParser
from Bio.SeqUtils import seq1

from tqdm.auto import tqdm

/datapool/data2/home/jiahan/anaconda3/envs/foldy/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from Bio.PDB import PDBParser, PDBIO

def merge_pdb_files(file1, file2, output_file):
    parser = PDBParser()
    io = PDBIO()

    structure1 = parser.get_structure("protein1", file1)
    structure2 = parser.get_structure("protein2", file2)

    # 获取第一个结构的第一个模型
    model1 = structure1[0]

    # 将第二个结构的所有链添加到第一个结构的第一个模型中
    for model in structure2:
        for chain in model:
            model1.add(chain)

    # 将合并后的结构写入新的 PDB 文件
    io.set_structure(structure1)
    io.save(output_file)

# 使用函数
merge_pdb_files('/datapool/data2/home/jiahan/Res Proj/PepDiff/frame-flow/misc/process/1a1r_C/peptide.pdb', 
                '/datapool/data2/home/jiahan/Res Proj/PepDiff/frame-flow/misc/process/1a1r_C/pocket.pdb', 
                '/datapool/data2/home/jiahan/Res Proj/PepDiff/frame-flow/misc/process/1a1r_C/pocket_merge.pdb')

In [3]:
input_dir = "/datapool/data2/home/jiahan/Data/PepMerge_new/"

In [4]:
tmps = []
for name in tqdm(os.listdir(input_dir)):
    try:
        merge_pdb_files(os.path.join(input_dir, name, 'pocket.pdb'),os.path.join(input_dir, name, "peptide.pdb"),os.path.join(input_dir, name, 'pocket_merge.pdb'))
        merge_pdb_files(os.path.join(input_dir, name, 'receptor.pdb'),os.path.join(input_dir, name, "peptide.pdb"),os.path.join(input_dir, name, 'receptor_merge.pdb'))
    except:
        tmps.append(name)

  0%|          | 4/10348 [00:00<12:09, 14.18it/s]

 11%|█         | 1089/10348 [01:22<12:37, 12.22it/s]/datapool/data2/home/jiahan/anaconda3/envs/foldy/lib/python3.9/site-packages/Bio/PDB/PDBParser.py:236: PDBConstructionWarning: Negative occupancy in one or more atoms
  warnings.warn(
 98%|█████████▊| 10169/10348 [13:45<00:14, 12.13it/s]/datapool/data2/home/jiahan/anaconda3/envs/foldy/lib/python3.9/site-packages/Bio/PDB/PDBParser.py:236: PDBConstructionWarning: Negative occupancy in one or more atoms
  warnings.warn(
100%|██████████| 10348/10348 [13:59<00:00, 12.32it/s]


In [5]:
tmps

['7yxw_1_III_A',
 '1cq4_1_III_B2',
 '7a8u_1_III_B1',
 '8a9l_1_III_C3',
 '8ejl_1_III_E2',
 '8et3_1_III_N',
 '7s4q_3_III_F2',
 '8dtu_1_III_A',
 '6b0x_3_III_L2',
 '6ghl_1_III_B',
 '5w5c_1_III_E1',
 '1f8v_4_III_E2',
 '8a93_1_III_C',
 '6zfm_1_III_C',
 '8bft_1_III_B2',
 '7s4q_3_III_E2',
 '1f8v_3_III_G3',
 '6b0x_4_III_L2',
 '3bzl_3_III_A',
 '7xva_1_III_B2',
 '1cq4_3_III_B3',
 '7oko_1_III_W',
 '3kqi_2_III_B2',
 '6r2i_1_III_B1',
 '8gqa_1_III_B2',
 '7pd3_1_III_UA',
 '7wmc_1_III_B',
 '6b0x_3_III_K2',
 '3fdo_3_III_B2',
 '6k3f_1_III_D',
 '8etc_1_III_PA',
 '6b0x_3_III_H2',
 '6lss_1_III_A',
 '7al2_1_III_B1',
 '7nkk_1_III_A',
 '7wul_1_III_A',
 '6hu4_3_III_G2',
 '1cq4_2_III_B5',
 '6kpb_1_III_B2',
 '1fpn_6_III_D1',
 '1nu7_1_III_A',
 '6hu4_4_III_G2',
 '6hu4_4_III_H2',
 '6hu4_3_III_H2',
 '7al2_1_III_B2',
 '2xbo_4_III_D2',
 '2bbv_6_III_E1',
 '3bzo_2_III_A1',
 '3fdo_3_III_B1',
 '6b0x_4_III_K2',
 '1xmn_1_III_E',
 '1f8v_3_III_E2',
 '8oxm_1_III_A',
 '6r2i_1_III_B2',
 '4q96_3_III_C',
 '6tmi_1_III_A',
 '6b0x_4_I

In [1]:
name = "6b0x_4_III_M2"

In [2]:
with open("/datapool/data2/home/jiahan/Res Proj/PepDiff/frame-flow/Data/names.txt",'r') as f:
    names = []
    for line in f:
        names.append(line.strip())

In [5]:
names

['1r1s_B',
 '8dgo_1_III_C',
 '1jyr_L',
 '3mxy_L',
 '2h5k_C',
 '1gbq_B',
 '1zfp_I',
 '1bm2_L',
 '5cdw_f',
 '1tze_I',
 '2gbq_B',
 '3n7y_D',
 '5cdw_I',
 '2vwf_B',
 '1gbr_B',
 '1uj0_B',
 '5gjh_B',
 '5ixf_B',
 '4cc3_B',
 '4cc7_B',
 '4cc2_B',
 '4lnp_B',
 '2sem_C',
 '1sem_C',
 '3sem_C',
 '1r1p_E',
 '3gbq_B',
 '1r1q_C',
 '1fyr_I',
 '1bmb_I',
 '4gbq_B',
 '3mxc_L',
 '3n8m_B',
 '3n84_G',
 '2w0z_B',
 '1aze_B',
 '5nsc_C',
 '5dj0_C',
 '6n9t_E',
 '6ifj_C',
 '5djx_C',
 '1dn2_E',
 '5di8_C',
 '5djc_C',
 '5dvn_B',
 '5djd_C',
 '5dvm_B',
 '5dk2_C',
 '5dj8_C',
 '6iqg_C',
 '5djz_C',
 '5dk0_C',
 '6iqh_C',
 '5dj6_C',
 '5dja_C',
 '5dvl_B',
 '5dvk_B',
 '5djy_C',
 '6iqg_D',
 '5dj2_C',
 '5u66_B',
 '3wnh_C',
 '4y1c_C',
 '3wne_C',
 '4y1d_D',
 '3avb_X',
 '3avi_D',
 '3avn_G',
 '3avj_D',
 '3avl_F',
 '3avm_D',
 '3ava_X',
 '3avh_E',
 '3avf_D',
 '3avg_D',
 '3avc_D',
 '6ex9_B',
 '3wnf_C',
 '3wng_C',
 '3av9_X',
 '3avk_D',
 '7nef_1_III_O',
 '6q86_C',
 '6q6x_E',
 '6q79_E',
 '7nef_1_III_J',
 '7nef_1_III_P',
 '6s7g_F',
 '7nef_1